In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
shakespeare_text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [5]:
tokenizer.texts_to_sequences(['first'])

[[20, 6, 9, 8, 3]]

In [6]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [7]:
max_id = len(tokenizer.word_index)

In [8]:
dataset_size = tokenizer.document_count

In [9]:
max_id

39

In [10]:
dataset_size

1115394

In [11]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [12]:
encoded.shape

(1115394,)

In [13]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [14]:
train_size

1003854

In [15]:
n_steps = 100

In [16]:
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [17]:
dataset = dataset.flat_map(lambda window : window.batch(window_length))

In [18]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows : (windows[:, :-1], windows[:, 1:]))

In [19]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [20]:
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=.2, recurrent_dropout=.2),
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=.2, recurrent_dropout=.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
history = model.fit(dataset, epochs=20)

Epoch 1/20
  23545/Unknown - 3528s 150ms/step - loss: 1.4876